# 3_0 FoldTree

In [1]:
# 初始化pyrosetta，使用pyrosetta必须要做的一件事情
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
init()

PyRosetta-4 2021 [Rosetta PyRosetta4.Release.python37.mac 2021.07+release.c48be2695c4ba637c6fa19ee5d289fd9a8aa99ef 2021-02-21T11:50:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python37.mac r275 2021.07+release.c48be2695c4 c48be2695c4ba637c6fa19ee5d289fd9a8aa99ef http://www.pyrosetta.org 2021-02-21T11:50:06
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/showmaker/miniconda3/envs/deepcadd/lib/python3.7/site-packages/pyrosetta-2021.7+release.c48be2695c4-py3.7-macosx-10.9-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-1219100168 seed_offset=0 real_seed=-1219100168
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-1219100168 RG_type=mt19937


在1_5中，我们学习了有关蛋白质构象几何的处理方式的各种概念及处理方式。在处理蛋白质构象设计的过程中，我们需要针对这些几何关系对蛋白质结构进行计算。此时，就会涉及到了计算过程中的坐标的处理方式。这一章将介绍的就是Rosetta中坐标的处理方式。

## 内坐标及自由度计算复杂度降低处理方式

PDB文件中，大多数的表示都是欧式坐标(X,Y,Z)，当改变或设置这3个值时，该原子的位置也就确定了，

此时每个原子都具有三个自由度，但三个自由度对于计算来说复杂度过高。为了便于计算在进行计算的时候，坐标的表示方式是这样的：

    （1）使用了内坐标来进行表示。内坐标定义如下：
        如下图所示，对于一个原子，其位置坐标通过其与临近原子的键长、键角、二面角来进行表示。此个原子来说，构象的改变是通过改变键长、键角和二面角来完成的，自由度仍为3

<img src="./imgs/internal_coordinate.png" width = "600" height = "300" align=center /> 

    (2) 减少自由度计算复杂度：
        尽管此时对于一个原子来说自由度仍为3，但这三个自由度在蛋白质设计中的重要程度并不相同。往往在蛋白质建模的过程中，键长、键角的变化相对来说设计很微小的，这时候我们设置该两者是确定的，以二面角的变化来表示构象变化。此时自由度就降为1了
        同时，对于特定的原子（例如大多数的氢）其二面角受其化学环境的影响是固定的，此时自由度就变为0了。

在下面这个路径中，我们可以找到Rosetta中关于每种天然氨基酸的参数文件

## FoldTree

### 杠杆效应

在经过以上基于内坐标的定义后，我们便可以改变蛋白质的二面角来改变蛋白质的构象。下面我们举例，首先得到了一个由五个残基组成的多肽，并更改了3号残基的phi角。

In [2]:
pose = pose_from_sequence('KPALN') # 根据序列生成一个示例的多肽序列
pose.dump_pdb('./data/example.pdb') # 保存该pose为pdb文件

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.730176 seconds.


True

In [3]:
# 更改示例多肽序列的3号残基的phi角
pose.set_phi(3,70) # 修改3号残基的phi角
pose.dump_pdb('./data/example_change.pdb')  # 保存修改后的pose为pdb文件

True

类似的，如下图，实际上只想动的是三号残基的phi角，但实际上，肽链的残基彼此相关，最后肽链的整个构象都被改变了，从第三个残基之后，肽链部分如同杠杆一般抬了起来。这便是杠杆效应。三号残基之前的残基都没有发生改变，而下游的残基会适应3号残基的改变来进行改变，可以理解为，这个改变只会调整3号残基下游的残基，上游则不会改变，而这便是FoldTree。

值得注意的是，以一个自由度的改变，表示了3N（下游）*atom_number个自由度的卡迪尔坐标换算，这种计算效率是极高的。

<img src="./imgs/phi_change.png" width = "600" height = "300" align=center /> 

### FoldTree 简介

FoldTree是处理给定结构中主链氨基酸连接方式的一种方法，就如上一小节所示，在Rosseta中对于蛋白质主链内部及主链之间会有一个上下游关系的表示，他定义了在蛋白质结构中，哪些残基是上游或是母节点、哪些残基是下游或是子节点，并为其贴上了标签。在Rosetta中规定，只有上游的氨基酸会发生改变，而下游不会发生。

如图所示，图中的多肽按照氨基酸的顺序贴上了标签来进行表示。当想要更改3号残基的二面角时，3号残基上游的残基会是刚性不会改变，而下游会随之改变。

<img src="./imgs/foldtree.png" width = "600" height = "300" align=center /> 

FoldTree除了上文提到过的杠杆效应外，还有两种特性：
<br>（1）顺序性</br>
<br>（2）杠杆效应</br>
<br>（3）跳跃性</br>  

## FoldTree的表示方式

Edge是FoldTree的基本组成部分，一个FoldTree是由1个或多个Edge组成。在Pyrosetta中Fold_Tree通过fold_tree方法来进行调用

In [4]:
# PDB:1v74的例子
pose_1v74 = pose_from_pdb('./data/1v74.pdb') #读入蛋白质的pdb文件
pose_1v74_foldtree = pose_1v74.fold_tree() #得到该pose的foldtree实例
print("\n1v74 Fold Tree:\n", pose_1v74_foldtree) #输出PDB:1V74的fold tree表示

core.import_pose.import_pose: File './data/1v74.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Loading (but possibly not actually using) '1PE' from the PDB components dictionary for residue type 'pdb_1PE'
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 107
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 195

1v74 Fold Tree:
 FOLD_TREE  EDGE 1 107 -1  EDGE 1 108 1  EDGE 1 109 2  EDGE 109 195 -1 


FoldTree的表示方式一般如下。有四个字段组成：

如同上面的输出所示，示例pose具有两个共价连接的链，分别是从1号残基到108号残基；以及109号残基到195号残基。

当Define字段是-1时，代表着这一段多肽区域按照从start到end位的顺序通过共价键进行延伸。如上所示，表示1到107残基、109到195残基是以共价相连。

当Define字段是正整数时，代表着会在start和end位氨基酸建立虚拟链接,代表这个是一个Jump点，表示在FoldTree中，第start号氨基酸和第end号氨基酸之间将建立虚拟链接，Jump点会直接改变不同多肽链之间的上下游的关系。如上所示，分别在1和108残基、1和109残基有两个jump点。

对于FoldTree的Jump，我们可以通过foldtree实例的一些方法得到他的信息。

In [5]:
print(f"Whether the pose's fold exists? {pose_1v74_foldtree.check_fold_tree()}") # 这个pose中是否存在foldtree
print(f"The size of the foldtree (number of edges) is {pose_1v74_foldtree.size()}") #Foldtree 的尺寸（edge的个数）
print(f"The number of the jump is {pose_1v74_foldtree.num_jump()}") #Foldtree 的jump点的个数

Whether the pose's fold exists? True
The size of the foldtree (number of edges) is 4
The number of the jump is 2


## FoldTree的顺序性

在FoldTree中,Start和End字段是顺序敏感的，包括N->C顺序以及C->N顺序。不同顺序，代表EDGE内氨基酸所对应的上下游关系不同。

还是以原本的的五肽为例，原本是以N-C的顺序来进行排列的。我们将其人为修改为C-N后，由于上下游的改变，修改二面角后构象造成的变化也不再相同

In [6]:
#观察原本五肽的FoldTree
pose = pose_from_sequence('KPALN') # 根据一个序列得到一个五肽序列的pose
pose.dump_pdb('./data/example_5_1.pdb') #保存五肽的构象
print("The Fold Tree of the pose:", pose.fold_tree()) # 输出该pose的FoldTree
print("Whether the foldtree exists?:", pose.fold_tree().check_fold_tree())# 判断该pose的FoldTree是否存在
# 设置一个foldtree对象的实例，并将它读回pose中更新foldtree
ft = FoldTree() #设置一个空的foldtree对象
ft.add_edge(5, 1, -1) #增加一个5号残基到1号残基的共价连接的edge
pose.fold_tree(ft) #更新pose的fold_tree
print(pose.fold_tree()) #输出foldtree
pose.set_phi(3,70) #更改肽链的3号残基的phi角
pose.dump_pdb('./data/example_5_1_change.pdb') #保存更改后的五肽的构象

The Fold Tree of the pose: FOLD_TREE  EDGE 1 5 -1 
Whether the foldtree exists?: True
FOLD_TREE  EDGE 5 1 -1 


True

<img src="./imgs/foldtree_sequence.png" width = "600" height = "300" align=center /> 

如图，黄色的是初始的五肽，蓝色的是原本的多肽在修改3号残基的phi角后的多肽构象，可以发现出现构象改变的是4、5号残基；而在修改该多肽的FoldTree为从5到1的顺序后，发生构象变化的就是1、2号残基。而这，就是foldtree的顺序性，即在改变选定残基的构象时，随之而改变的只有选定残基下游的氨基酸的构象

关于foldtree的Jump点和跳跃性，将会放到下一节来进行讲解。

## 自定义或修改FoldTree

在上面讲解FoldTree的顺序性的时候，其实我们已经完成了相应的操作，最基本操作方式便是为期增加或者删除edge如下：

（1）实例化一个FoldTree对象;  
（2）使用该对象的add_edge、delete_edge进行FoldTree的编辑;  
（3）pose的fold_tree读回创建的FoldTree对象。

In [7]:
pose_1v74 = pose_from_pdb('./data/1v74.pdb') #读入PDB文件
print(pose_1v74.fold_tree())
ft = FoldTree()# 实例化一个FoldTree对象
ft.add_edge(1, 107, -1) # 加入一个start：1，end 107，且是共价连接的edge
ft.add_edge(107, 108, 1) # 加入一个start：107，end 108，且定义为Jump的edge
ft.add_edge(108, 195, -1) # 加入一个start：108，end 195，且是共价连接的的edge
pose_1v74.fold_tree(ft) # 将创建的FoldTree对象用fold_tree方法读回去
print(pose_1v74.fold_tree()) #输出现在的FoldTree

core.import_pose.import_pose: File './data/1v74.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 107
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 195
FOLD_TREE  EDGE 1 107 -1  EDGE 1 108 1  EDGE 1 109 2  EDGE 109 195 -1 
FOLD_TREE  EDGE 1 107 -1  EDGE 107 108 1  EDGE 108 195 -1 


可以看到，我们修改了示例蛋白的FoldTree，没有FoldTree的序列也可以通过相同的方式自定义FoldTree